In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai import *
from fastai.vision import *
import numpy as np # linear algebra
import pandas as pd 

In [3]:
import os
path = '../input'
print(os.listdir(path))

FileNotFoundError: ignored

In [0]:

class CustomImageList(ImageList):
    def open(self, fn):
        img = fn.reshape(28,28)
        img = np.stack((img,)*3, axis=-1)
        return Image(pil2tensor(img, dtype=np.float32))
    
    @classmethod
    def from_csv_custom(cls, path:PathOrStr, csv_name:str, imgIdx:int=1, header:str='infer', **kwargs)->'ItemList': 
        df = pd.read_csv(Path(path)/csv_name, header=header)
        res = super().from_df(df, path=path, cols=0, **kwargs)
        
        res.items = df.iloc[:,imgIdx:].apply(lambda x: x.values / 255.0, axis=1).values
        
        return res


In [0]:
test = CustomImageList.from_csv_custom(path=path, csv_name='test.csv', imgIdx=0)

In [0]:
tfms = get_transforms(do_flip=False)
data = (CustomImageList.from_csv_custom(path=path, csv_name='train.csv', imgIdx=1)
                .split_by_rand_pct(.2)
                .label_from_df(cols='label')
                .add_test(test, label=0)
                .transform(tfms)
                .databunch(bs=128, num_workers=0)
                .normalize(imagenet_stats))

In [0]:
data.show_batch(rows=3, figsize=(5,5))


In [0]:
#learn = cnn_learner(data, models.resnet34, metrics=accuracy, model_dir='/kaggle/working/models')
learn = cnn_learner(data, models.resnet50, metrics=accuracy, model_dir='/kaggle/working/models')

In [0]:
learn.fit_one_cycle(4)

In [4]:
learn.save('stage-1-resnet50')

NameError: ignored

In [5]:
learn.load('stage-1-resnet50')

NameError: ignored

In [6]:
learn.lr_find()


NameError: ignored

In [0]:
learn.recorder.plot()

In [0]:
learn.unfreeze()
learn.fit_one_cycle(12, max_lr=slice(1e-6,1e-3))

In [0]:
learn.save('stage-1-resnet34')

In [0]:
learn.load('stage-1-resnet34')
learn.validate()

In [0]:
learn.lr_find()
learn.recorder.plot()

In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot()

In [0]:
learn.unfreeze()
learn.fit_one_cycle(12, max_lr=slice(7e-6,1e-4))

In [0]:
learn.save('stage-1-resnet34')

In [0]:
learn.load('stage-1-resnet34')
learn.validate()

In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot()

In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot()

In [0]:
learn.unfreeze()
learn.fit_one_cycle(12, max_lr=slice(1e-6,1e-4))